# Extract Churn Data 


In [1]:
# Data Handling
import pandas as pd
import numpy as np

# Data Imputing
from sklearn.impute import KNNImputer

# Modelling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings("ignore")

import requests
from io import StringIO
import re

# Import des librairies nécessaires
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import joblib
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime


OSError while attempting to symlink the latest log directory


In [2]:
# #Dataset under apache license: https://github.com/IBM/telco-customer-churn-on-icp4d/blob/master/LICENSE
# csv = requests.get("https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv").text
# df = pd.read_csv(StringIO(csv), sep=",")
# raw_data_path = "../data"
# df.to_csv('../data/raw_data.csv', index=False)

# Bronze table name for advanced
# bronze_table_name = "advanced_churn_bronze_customers"


In [ ]:




def save_data(df):
    df.to_csv('../data/raw_data.csv', index=False)
    print("Données sauvegardées")

# 1. Fonction pour extraire les données et les sauvegarder
def extract_data():
    url = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"  # Remplacer par ton URL
    response = requests.get(url)
    csv = response.text  # Ou response.text si c'est du texte brut
    df = pd.read_csv(StringIO(csv), sep=",")
    # 
    print("Données extraites ")
    return df 

def extract_save():
    df = extract_data()
    save_data(df) 


def cleanup_column(pdf):

    # Clean up column names
    pdf.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower().replace("__", "_") for name in pdf.columns]
    pdf.columns = [re.sub(r'[\(\)]', '', name).lower() for name in pdf.columns]
    pdf.columns = [re.sub(r'[ -]', '_', name).lower() for name in pdf.columns]
    
    return pdf.rename(columns = {'streaming_t_v': 'streaming_tv', 'customer_i_d': 'customer_id'})

def prepare_data(df):
    df = cleanup_column(df)

    # Transformer la variable 'churn' (Yes/No) en 0/1
    df['churn'] = pd.to_numeric(df['churn'].replace({'Yes': 1, 'No': 0}), errors='coerce')

    # Garder uniquement les colonnes numériques
    df_numeric =  df.select_dtypes(include=['number'])

    # Supprimer les valeurs manquantes (NA) dans les colonnes numériques
    df_clean = df_numeric.dropna() 
    # ... 
    return df_clean 

def prepare_save():
    df = pd.read_csv('../data/raw_data.csv')
    df_clean = prepare_data(df)
    df_clean.to_csv('../data/cleaned_data.csv', index=False) 
    print("Données préparées et sauvegardées")

# 3. Fonction pour entraîner un modèle simple
def train_model():
    df = pd.read_csv('../data/cleaned_data.csv')
    X = df.drop('churn', axis=1)  # Remplacer 'target' par le nom de la variable cible
    y = df['churn']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Sauvegarder le modèle
    joblib.dump(model, '../models/logistic_model.pkl')
    print("Modèle entraîné et sauvegardé")
    
extract_save()
prepare_save()



train_model()


Données extraites 
Données sauvegardées
Données préparées et sauvegardées


In [ ]:
# df = pd.read_csv('../data/cleaned_data.csv')
# df["churn"]

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: churn, Length: 7043, dtype: int64

Modèle entraîné et sauvegardé
